In [1]:
# Import packages
from dash import Dash, html, dash_table, dcc, callback, Output, Input, State
import pandas as pd
import plotly.express as px
import copy
import numpy as np
from mlflow.sklearn import load_model


In [2]:
# On load les données
data = pd.read_csv("concatenate_files.csv")

In [3]:
data["ANNEES_AGE"] = (abs(data.DAYS_BIRTH) / 365.25)
data["ANNEES_LAST_PHONE_CHANGE"] = round((abs(data.DAYS_LAST_PHONE_CHANGE) / 365.25), 2)

# On élimine les anciennes variables
data.drop(["DAYS_BIRTH", "DAYS_LAST_PHONE_CHANGE"], axis = 1, inplace = True)

In [4]:
infinity_indices = np.where(np.isinf(data))
for row_ind, col_ind in zip(*infinity_indices):
    data.iloc[row_ind, col_ind] = data.iloc[:,col_ind].median()

In [5]:
# On load le modèle 
model = load_model("banking_model_20230901135647")

In [6]:
data_dashboard = model[0].transform(data)

In [7]:
data_dashboard

,SK_ID_CURR,AMT_INCOME_TOTAL,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_REGISTRATION,FLAG_MOBIL,CNT_FAM_MEMBERS,LIVE_REGION_NOT_WORK_REGION,REG_CITY_NOT_LIVE_CITY,...,APPROVED_RATE_DOWN_PAYMENT_MEAN,APPROVED_DAYS_DECISION_MAX,POS_MONTHS_BALANCE_MAX,POS_MONTHS_BALANCE_MEAN,POS_NAME_CONTRACT_STATUS_XNA_MEAN,INSTAL_PAYMENT_DIFF_MAX,INSTAL_DAYS_ENTRY_PAYMENT_MAX,INSTAL_DAYS_ENTRY_PAYMENT_MEAN,INSTAL_COUNT,ANNEES_AGE
0,100002,202500.0,24700.5,351000.0,0.018801,-3648.0,1,1.0,0,0,...,0.000000,-606.0,-1.0,-10.000000,0.0,0.000,-49.0,-315.421053,19.0,25.902806
1,100003,270000.0,35698.5,1129500.0,0.003541,-1186.0,1,2.0,0,0,...,0.050030,-746.0,-18.0,-43.785714,0.0,0.000,-544.0,-1385.320000,25.0,45.900068
2,100004,67500.0,6750.0,135000.0,0.010032,-4260.0,1,1.0,0,0,...,0.212008,-815.0,-24.0,-25.500000,0.0,0.000,-727.0,-761.666667,3.0,52.145106
3,100006,135000.0,29686.5,297000.0,0.008019,-9833.0,1,2.0,0,0,...,0.163412,-181.0,-1.0,-9.619048,0.0,0.000,-12.0,-271.625000,16.0,52.032854
4,100007,121500.0,21865.5,513000.0,0.028663,-4311.0,1,1.0,0,0,...,0.159516,-374.0,-1.0,-33.636364,0.0,22655.655,-14.0,-1032.242424,66.0,54.570842
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
307502,456251,157500.0,27558.0,225000.0,0.032561,-8456.0,1,1.0,0,0,...,0.000000,-273.0,-1.0,-5.000000,0.0,0.000,-38.0,-156.285714,7.0,25.535934
307503,456252,72000.0,12001.5,225000.0,0.025164,-4388.0,1,1.0,0,0,...,0.062443,-2497.0,-76.0,-79.000000,0.0,0.000,-2327.0,-2393.833333,6.0,56.878850
307504,456253,153000.0,29979.0,585000.0,0.005002,-6737.0,1,1.0,0,0,...,0.214316,-1909.0,-57.0,-79.235294,0.0,3945.825,-1738.0,-2387.428571,14.0,40.974675
307505,456254,171000.0,20205.0,319500.0,0.005313,-2562.0,1,2.0,0,1,...,0.000000,-277.0,-1.0,-5.550000,0.0,0.000,-18.0,-161.263158,19.0,32.747433


In [8]:
data_dashboard.shape

(307507, 100)

In [9]:
(model.predict_proba(data_dashboard)[:, 1] >= (0.42)).astype(int)

array([1, 0, 0, ..., 0, 1, 1])

In [10]:
data["prediction"] = (model.predict_proba(data)[:, 1] >= (0.42)).astype(int)
data["prediction_pret"] = data["prediction"].apply(lambda x: "Pret" if x == 0 else "Non pret")
data["proba_pred_pret"] = model.predict_proba(data)[:, 0]
data["proba_pred_non_pret"] = model.predict_proba(data)[:, 1]

In [30]:
cols = [1] + list(range(-4, 0))

In [31]:
df.iloc[:, cols]

,AMT_INCOME_TOTAL,prediction,prediction_pret,proba_pred_pret,proba_pred_non_pret
0,202500.0,1,Non pret,0.28,0.72
1,270000.0,0,Pret,0.93,0.07
2,67500.0,0,Pret,0.71,0.29
3,135000.0,0,Pret,0.73,0.27
4,121500.0,0,Pret,0.63,0.37
...,...,...,...,...,...
95,76500.0,1,Non pret,0.45,0.55
96,135000.0,1,Non pret,0.17,0.83
97,90000.0,1,Non pret,0.49,0.51
98,243000.0,1,Non pret,0.47,0.53


In [14]:
df = data_dashboard[:100]

In [125]:
selected = 1

if type(selected) != list:
    selected = [selected]
selected

<class 'int'>


[1]

In [108]:
df[df['SK_ID_CURR'].isin([100002, 100003])]

,SK_ID_CURR,AMT_INCOME_TOTAL,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_REGISTRATION,FLAG_MOBIL,CNT_FAM_MEMBERS,LIVE_REGION_NOT_WORK_REGION,REG_CITY_NOT_LIVE_CITY,...,POS_NAME_CONTRACT_STATUS_XNA_MEAN,INSTAL_PAYMENT_DIFF_MAX,INSTAL_DAYS_ENTRY_PAYMENT_MAX,INSTAL_DAYS_ENTRY_PAYMENT_MEAN,INSTAL_COUNT,ANNEES_AGE,prediction,prediction_pret,proba_pred_pret,proba_pred_non_pret


In [93]:
options=[{'label': str(client), 'value': client} for client in df['SK_ID_CURR']]
options.append({'label': None, 'value': None})
options

[{'label': '100002', 'value': 100002},
 {'label': '100003', 'value': 100003},
 {'label': '100004', 'value': 100004},
 {'label': '100006', 'value': 100006},
 {'label': '100007', 'value': 100007},
 {'label': '100008', 'value': 100008},
 {'label': '100009', 'value': 100009},
 {'label': '100010', 'value': 100010},
 {'label': '100011', 'value': 100011},
 {'label': '100012', 'value': 100012},
 {'label': '100014', 'value': 100014},
 {'label': '100015', 'value': 100015},
 {'label': '100016', 'value': 100016},
 {'label': '100017', 'value': 100017},
 {'label': '100018', 'value': 100018},
 {'label': '100019', 'value': 100019},
 {'label': '100020', 'value': 100020},
 {'label': '100021', 'value': 100021},
 {'label': '100022', 'value': 100022},
 {'label': '100023', 'value': 100023},
 {'label': '100024', 'value': 100024},
 {'label': '100025', 'value': 100025},
 {'label': '100026', 'value': 100026},
 {'label': '100027', 'value': 100027},
 {'label': '100029', 'value': 100029},
 {'label': '100030', 'val

In [130]:
# Initialize the app - incorporate css
external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']
app = Dash(__name__, external_stylesheets=external_stylesheets)


# App layout
app.layout = html.Div([
    
    # Titre dashboard
    html.Div(className='row', children="Utilisation du modèle pour prédire l'obtention d'un prêt",
             style={'textAlign': 'center', 'color': 'black', 'fontSize': 30}),
    
    html.Div(className='row', children=[
        html.Div(
            className='client-selection',  # Add a class for styling
            children=[
                html.Div("Choix d'un client :",
                         style={'textAlign': 'left', 'color': 'black', 'fontSize': 18}),
                dcc.Dropdown(
                    options=[{'label': str(client), 'value': client} for client in df['SK_ID_CURR']],
                    value=None,
                    style={'textAlign': 'left', 'color': 'black', 'fontSize': 15},
                    placeholder="Sélectionne un client",
                    multi=True,
                    id='dropdown'
                ),
            ],
        ),
        html.Div(id="table_client")
    ]),

    
    # Boutton pour la prédiction
    html.Div(style={'display': 'flex', 'justifyContent': 'center', 'alignItems': 'center', 'height': '20vh'},
        children=[html.Div(style={'textAlign': 'center', 'color': 'black', 'fontSize': 18},
                children=[html.Button("Prédiction", id="prediction_button", n_clicks=0)],
                )]
            ),
    
    # Tableau à afficher
    html.Div(id="table_div"),    

])

# On crée une fonction callback afin de mettre à jour le boutton 
@app.callback(
    Output("table_client", "children"),
    Output("prediction_button", "children"),
    Output("prediction_button", "disabled"),
    Output("table_div", "children"),
    Input("dropdown", "value"),
    Input("prediction_button", "n_clicks"),
    State("prediction_button", "children")
)

def update_table_and_button(selected_client, n_clicks, button_label):
    # Remise à 0
    if selected_client == []:
        selected_client = None
    
    if (type(selected_client) != list) & (selected_client is not None) :
        selected_client = [selected_client]
        
    if selected_client is None :
        table1 = dash_table.DataTable(
            data=df.to_dict('records'),
            page_size=10,
            style_table={'overflowX': 'auto'}
        )
    else :
        filtered_df = df[df['SK_ID_CURR'].isin(selected_client)]
        table1 = dash_table.DataTable(
            data=filtered_df.to_dict('records'),
            page_size=10,
            style_table={'overflowX': 'auto'}
        )
    
    # Par défaut
    if n_clicks is None :
        n_clicks = 0
    if n_clicks == 0 :
        button_label = "Faire la prédiction"
        button_disabled = False
        table2 = None
    else:
        button_label = "Prédiction réalisée"
        button_disabled = True
        # Your code to update the table here (replace with your actual code)
        # ...
        cols = [0] + list(range(-4, 0))
        
        if selected_client is None :
            table2 = dash_table.DataTable(
                data=df.iloc[:, cols].to_dict('records'),
                page_size=10,
                style_table={'overflowX': 'auto'}
            )
        else :
            filtered_df = df[df['SK_ID_CURR'].isin(selected_client)]
            table2 = dash_table.DataTable(
                data=filtered_df.iloc[:, cols].to_dict('records'),
                page_size=10,
                style_table={'overflowX': 'auto'}
            )
        
    return table1, button_label, button_disabled, table2

# Run the app
if __name__ == '__main__':
    app.run(debug=True)

In [ ]:
# Incorporate data
df = copy.deepcopy(data_dashboard[:100])

# Initialize the app - incorporate css
external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']
app = Dash(__name__, external_stylesheets=external_stylesheets)


# App layout
app.layout = html.Div([
    html.Div(className='row', children='My First App with Data, Graph, and Controls',
             style={'textAlign': 'center', 'color': 'black', 'fontSize': 30}),
    
    # Variables à afficher
    html.Div(className='row', children=[
        dcc.RadioItems(options=['DAYS_EMPLOYED_PERC', 'AMT_INCOME_TOTAL'],
                       value='AMT_INCOME_TOTAL',
                       inline=True,
                       style={'textAlign': 'left', 'color': 'black', 'fontSize': 15},
                       id='variables_affichage_buttons')
    ]),
    
    # Options d'affichage
    html.Div(className='row', children=[
        html.Label("Taille des axes et valeurs :"),
        dcc.Slider(
        id='font-size-slider',
        min=18,
        max=42,
        step=6,
        value=18,
        marks={i: str(i) for i in range(18, 43, 6)}),
        
        # Options for the color palette dropdown
        html.Label("Select Color Palette:"),
        dcc.RadioItems(
            id='color-palette-dropdown',
            options=[
                {'label': 'Default', 'value': 'plotly'},
                {'label': 'Colorblind-Friendly', 'value': 'colorblind'}
               
            ],
            value='plotly'
        ),
    ]),
    
    # Graph à afficher
    html.Div(className='row', children=[
        dcc.Graph(figure={}, id='histo-chart-final')
    ]),
    
    # Tableau à afficher
    html.Div(className='row', children=[
        dash_table.DataTable(data=df.to_dict('records'), page_size=10, style_table={'overflowX': 'auto'})
    ])
])

# Add controls to build the interaction
@callback(
    Output(component_id='histo-chart-final', component_property='figure'),
    Input(component_id='variables_affichage_buttons', component_property='value'),
    Input(component_id='font-size-slider', component_property='value'),
    Input(component_id='color-palette-dropdown', component_property='value')
)
def update_graph(col_chosen, font_size, color_palette):
    
    x_var = "CODE_GENDER"
    fig = px.histogram(df, 
                       x=x_var, 
                       y=col_chosen, 
                       histfunc='avg')
    
    if color_palette == 'colorblind':
        # On utilise une palette pour les personnes avec une déficience de couleur (source : https://www.color-hex.com/color-palette/28639)
        color_blind_hex = ['#d6f9cc', '#97989c', '#33926e', '#1c4226', '#1f3628']
        fig.update_traces(marker_color=color_blind_hex)
        
    else:
        # Use the default Plotly color palette
        fig.update_traces(marker_color=px.colors.qualitative.Plotly)
    
    fig.update_layout(
        xaxis_title=x_var,
        yaxis_title=col_chosen,
        font=dict(
            family='Arial',
            size=font_size,
            color='black'),
        
        #xaxis_title_font=dict(size=font_size, family='Arial', color='black'),
        #yaxis_title_font=dict(size=font_size, family='Arial', color='black')
    )
    
    return fig

# Run the app
if __name__ == '__main__':
    app.run(debug=True)